In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

In [2]:
df_ords_prods_merge = pd.read_pickle(r'C:\Users\charan.lalwani\Documents\cf\Achievement 4\Instacart basket analysis\02 Data\Prepared data\ords_prods_merge.new.pkl') 

In [3]:
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean']})

,order_number
,mean
department_id,
1,15.457838
2,17.277920
3,17.170395
4,17.811403
5,15.215751
6,16.439806
7,17.225802
8,15.340650


# The subset is calculating mean values of only certain departments where as the entire dataframe is calculating the mean values for all the departments.

In [4]:
df_ords_prods_merge.groupby('department_id').agg({'order_number': ['mean', 'min', 'max']})

order_number        
                      mean min max
department_id                     
1                15.457838   1  99
2                17.277920   1  99
3                17.170395   1  99
4                17.811403   1  99
5                15.215751   1  99
6                16.439806   1  99
7                17.225802   1  99
8                15.340650   1  99
9                15.895474   1  99
10               20.197148   1  99
11               16.170638   1  99
12               15.887671   1  99
13               16.583536   1  99
14               16.773669   1  99
15               16.165037   1  99
16               17.665606   1  99
17               15.694469   1  99
18               19.310397   1  99
19               17.177343   1  99
20               16.473447   1  99
21               22.902379   1  99

In [5]:
df_ords_prods_merge['max_order'] = df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [6]:
pd.options.display.max_rows = None

In [6]:
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'
df_ords_prods_merge.loc[(df_ords_prods_merge['max_order'] <= 40) & (df_ords_prods_merge['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [7]:
df_ords_prods_merge['loyalty_flag'].value_counts(dropna=False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

In [8]:
df_ords_prods_merge.head()

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,...,product_name,aisle_id,department_id,prices,merge_indicator,busiest_day,busiest_days,busiest_period_of_day,max_order,loyalty_flag
0,2539329,1,prior,1,2,8,NaN,196,1,0,...,Soda,77,7,9.0,both,Regularly busy,Regularly busy,Most orders,10,New customer
1,2398795,1,prior,2,3,7,15.0,196,1,1,...,Soda,77,7,9.0,both,Regularly busy,Slowest days,Fewest orders,10,New customer
2,473747,1,prior,3,3,12,21.0,196,1,1,...,Soda,77,7,9.0,both,Regularly busy,Slowest days,Most orders,10,New customer
3,2254736,1,prior,4,4,7,29.0,196,1,1,...,Soda,77,7,9.0,both,Least busy,Slowest days,Fewest orders,10,New customer
4,431534,1,prior,5,4,15,28.0,196,1,1,...,Soda,77,7,9.0,both,Least busy,Slowest days,Most orders,10,New customer


#  Creating a spending flag for each user based on the average price across all their orders.

In [10]:
df_ords_prods_merge['spenders'] =df_ords_prods_merge.groupby(['user_id'])['order_number'].transform(np.max)

In [11]:
df_ords_prods_merge.loc[df_ords_prods_merge['spenders'] < 10 ,'Spending_Flag'] = 'Low spender'
df_ords_prods_merge.loc[df_ords_prods_merge['spenders'] >= 10 ,'Spending_Flag'] = 'High spender'

In [12]:
df_ords_prods_merge['Spending_Flag'].value_counts(dropna = False)

Spending_Flag
High spender    26933993
Low spender      5470866
Name: count, dtype: int64

#  Creating an order frequency flag that marks the regularity of a user’s ordering behavior according to the median in the “days_since_prior_order” column.

In [13]:
# Calculate the median of "days_since_prior_order" for each user
df_user_order_frequency = df_ords_prods_merge.groupby('user_id')['days_since_prior_order'].median().reset_index()
df_user_order_frequency.columns = ['user_id', 'median_days_since_prior_order']

In [14]:
# Merge the calculated median values back to the main DataFrame
df_ords_prods_merge = pd.merge(df_ords_prods_merge, df_user_order_frequency, on='user_id', how='left')

In [15]:
# Create the order frequency flag
df_ords_prods_merge['order_frequency_flag'] = 'Non-frequent customer'
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_prior_order'] > 10, 'order_frequency_flag'] = 'Regular customer'
df_ords_prods_merge.loc[df_ords_prods_merge['median_days_since_prior_order'] <= 10, 'order_frequency_flag'] = 'Frequent customer'

In [16]:
path = r"C:\Users\charan.lalwani\Documents\cf\Achievement 4\Instacart basket analysis\02 Data\Prepared data"
df_ords_prods_merge.to_pickle(os.path.join(path, 'ords_prods_merged.pkl'))

In [17]:
df_ords_prods_merge.to_pickle(os.path.join(path, 'ords_prods_merged.pkl'))